In [1]:
import os, textwrap

from openai import AzureOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langfuse import Langfuse

OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")


In [2]:
client = AzureOpenAI(api_version=OPENAI_API_VERSION)

In [3]:
lf = Langfuse(public_key=os.getenv("LANGFUSE_PUBLIC_KEY"),
              secret_key=os.getenv("LANGFUSE_SECRET_KEY"),
              host=os.getenv("LANGFUSE_HOST"))

In [4]:
stock_ticker_system_prompt = "you are a helpful financial data and stock market analysis assistant."

stock_ticker_user_prompt = textwrap.dedent("\nprovide the stock ticker symbol for the company in user {{query}}.\njust provide the stock ticker symbol, along with the company name, nothing else.\nif you can not figure out the stock ticker symbol, just return 'UNKNOWN'.\n the output should be a JSON object with first field company_name and second field, stock_code.")

In [5]:
lf.create_prompt(
    name='stock-ticker-chat-final',
    type='chat',
    prompt=[
      { "role": "system", "content": stock_ticker_system_prompt },
      { "role": "user", "content": stock_ticker_user_prompt },
    ],
    labels=['production'],
    config={"model": "gpt-4o-mini", "temperature": 0, "supported_language": ["en"]}
)

In [6]:
stock_ticker_prompt = lf.get_prompt("stock-ticker-chat-final")
stock_ticker_prompt.variables

['query']

In [7]:
stock_ticker_prompt.prompt

[{'role': 'system',
  'content': 'you are a helpful financial data and stock market analysis assistant.'},
 {'role': 'user',
  'content': "\nprovide the stock ticker symbol for the company in user {{query}}.\njust provide the stock ticker symbol, along with the company name, nothing else.\nif you can not figure out the stock ticker symbol, just return 'UNKNOWN'.\n the output should be a JSON object with first field company_name and second field, stock_code."}]

In [8]:

prompt_temp = ChatPromptTemplate.from_messages([
    (msg["role"], msg["content"].replace('{{query}}','{query}')) for msg in stock_ticker_prompt.prompt
])

model = AzureChatOpenAI(model='myllm')

parser = JsonOutputParser()

yahoo_news = YahooFinanceNewsTool()

def ticker_and_news(output_dict):
    news = yahoo_news.invoke(output_dict["stock_code"])
    return {"company_name": output_dict["company_name"], "stock_code": output_dict["stock_code"], "finance_news": news}

In [9]:
intermediate_chain = prompt_temp | model | parser | ticker_and_news

intermediate_chain.invoke({"query":"WHAT IS confluent?"})

{'company_name': 'Confluent, Inc.',
 'stock_code': 'CFLT',
 'finance_news': 'Buy 5 AI-Focused Mid-Cap Internet Software Stocks for a Solid Portfolio\nPATH, FIVN, FSLY, CALX and CFLT are five AI-focused mid-cap stocks that show rising earnings potential and momentum for 2025.\n\nIs There Now An Opportunity In Confluent, Inc. (NASDAQ:CFLT)?\nConfluent, Inc. ( NASDAQ:CFLT ), is not the largest company out there, but it saw a significant share price rise of 24...\n\n2 Reasons to Like CFLT (and 1 Not So Much)\nWhat a brutal six months it’s been for Confluent. The stock has dropped 24.6% and now trades at $24.48, rattling many shareholders. This might have investors contemplating their next move.'}

In [10]:
sentiment_analyzer_system_prompt = "You are a helpful financial news sentimaent analyzer."

sentiment_analyzer_user_prompt = """For the given input dict with {{company_name}}, {{stock_code}}, and {{finance_news}} containing news articles about a company.
                            For each news article in list extract the following as a JSON object with these fields:
                            'company_name', 'stock_code', 'news_description', 'sentiment', 'people_names',
                            'places_names', 'other_companies_referred', 'related_industries',
                            'market_implications', 'confidence_score'.
                            The field 'sentiment' should be one of 'positive', 'negative', or 'neutral'.
                            If a field is not present, use an empty string or empty list as appropriate.
                            Append the JSON objects for each news article and return the list."""

In [11]:
lf.create_prompt(
    name='sentiment-analyzer-chat-final',
    type='chat',
    prompt=[
      { "role": "system", "content": sentiment_analyzer_system_prompt },
      { "role": "user", "content": sentiment_analyzer_user_prompt },
    ],
    labels=['production'],
    config={"model": "gpt-4o-mini", "temperature": 0, "supported_language": ["en"]}
)

In [12]:
sentiment_analyzer_prompt = lf.get_prompt("sentiment-analyzer-chat-final")
sentiment_analyzer_prompt.variables

['company_name', 'stock_code', 'finance_news']

In [13]:
sentiment_analyzer_prompt.prompt

[{'role': 'system',
  'content': 'You are a helpful financial news sentimaent analyzer.'},
 {'role': 'user',
  'content': "For the given input dict with {{company_name}}, {{stock_code}}, and {{finance_news}} containing news articles about a company.\n                            For each news article in list extract the following as a JSON object with these fields:\n                            'company_name', 'stock_code', 'news_description', 'sentiment', 'people_names',\n                            'places_names', 'other_companies_referred', 'related_industries',\n                            'market_implications', 'confidence_score'.\n                            The field 'sentiment' should be one of 'positive', 'negative', or 'neutral'.\n                            If a field is not present, use an empty string or empty list as appropriate.\n                            Append the JSON objects for each news article and return the list."}]

In [14]:
sentiment_analyzer_prompt_temp = ChatPromptTemplate.from_messages([
    (
        msg["role"],
        msg["content"]
            .replace("{{company_name}}", "{company_name}")
            .replace("{{stock_code}}", "{stock_code}")
            .replace("{{finance_news}}", "{finance_news}")
    )
    for msg in sentiment_analyzer_prompt.prompt
])

In [15]:
final_chain = prompt_temp | model | parser | ticker_and_news | sentiment_analyzer_prompt_temp | model | parser

final_chain.invoke({"query":"WHAT IS confluent?"})

[{'company_name': 'Confluent, Inc.',
  'stock_code': 'CFLT',
  'news_description': 'Buy 5 AI-Focused Mid-Cap Internet Software Stocks for a Solid Portfolio. PATH, FIVN, FSLY, CALX and CFLT are five AI-focused mid-cap stocks that show rising earnings potential and momentum for 2025.',
  'sentiment': 'positive',
  'people_names': [],
  'places_names': [],
  'other_companies_referred': ['PATH', 'FIVN', 'FSLY', 'CALX'],
  'related_industries': ['AI', 'Internet Software'],
  'market_implications': 'Potential for solid portfolio growth',
  'confidence_score': 0.75},
 {'company_name': 'Confluent, Inc.',
  'stock_code': 'CFLT',
  'news_description': 'Confluent, Inc. (NASDAQ:CFLT), is not the largest company out there, but it saw a significant share price rise of 24...',
  'sentiment': 'positive',
  'people_names': [],
  'places_names': [],
  'other_companies_referred': [],
  'related_industries': [],
  'market_implications': 'Positive price movement indicates potential investor interest',
  'c